In [1]:
from torchvision.models import mobilenet_v3_small
from torchvision.models.mobilenetv3 import MobileNet_V3_Small_Weights
import torch.nn as nn
import torch

In [2]:
import torch
print(torch.cuda.is_available())


True


In [3]:
mobilenet_small = mobilenet_v3_small(pretrained=True)

/gris/gris-f/homestud/jplapper/anaconda3/envs/signenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/gris/gris-f/homestud/jplapper/anaconda3/envs/signenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
mobilenet = mobilenet_small.to(device)


cuda


In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)


In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the dataset path and set the batch size
dataset_path = '/gris/gris-f/homestud/jplapper/SignLanguageApp/SigNN Character Database'
batch_size = 64  # You can adjust this based on your system's memory


In [7]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [8]:
# Load the entire dataset
full_dataset = datasets.ImageFolder(dataset_path, transform=data_transforms['train'])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Now apply the appropriate transformations to each dataset split
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}


In [9]:
import torch.nn as nn
import torchvision.models as models

mobilenet = models.mobilenet_v3_small(pretrained=True)
for param in mobilenet.parameters():
    param.requires_grad = False

# Adjust the final layer according to the number of classes in your dataset
num_ftrs = mobilenet.classifier[3].in_features
NUM_CLASSES = 24
mobilenet.classifier[3] = nn.Linear(num_ftrs, NUM_CLASSES)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mobilenet = mobilenet.to(device)


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.classifier[3].parameters(), lr=0.001)

EPOCHS = 10

for epoch in range(EPOCHS):
    mobilenet.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = mobilenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Training loss: {total_loss/len(train_loader)}")

    # Validation accuracy
    mobilenet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = mobilenet(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Validation accuracy: {100.0 * correct / total}%")


Epoch 1/10, Training loss: 1.9616945374686763
Epoch 1/10, Validation accuracy: 65.89698046181172%
Epoch 2/10, Training loss: 1.0443335057429548
Epoch 2/10, Validation accuracy: 76.55417406749557%
Epoch 3/10, Training loss: 0.7579102966020692
Epoch 3/10, Validation accuracy: 84.84310242747188%
Epoch 4/10, Training loss: 0.6023597419261932
Epoch 4/10, Validation accuracy: 88.39550029603316%
Epoch 5/10, Training loss: 0.5032054753798358
Epoch 5/10, Validation accuracy: 89.40201302545886%
Epoch 6/10, Training loss: 0.43164440595878745
Epoch 6/10, Validation accuracy: 90.05328596802842%
Epoch 7/10, Training loss: 0.3787866455485236
Epoch 7/10, Validation accuracy: 91.53345174659562%
Epoch 8/10, Training loss: 0.33506086075081015
Epoch 8/10, Validation accuracy: 92.1255180580225%
Epoch 9/10, Training loss: 0.3084867177706844
Epoch 9/10, Validation accuracy: 91.88869153345175%
Epoch 10/10, Training loss: 0.280599260667585
Epoch 10/10, Validation accuracy: 92.36234458259325%


In [11]:
torch.save(mobilenet.state_dict(), 'asl_recognition_mobilenet.pth')


In [12]:
resnet101 = models.resnet101(pretrained=True)

# Freeze the model parameters
for param in resnet101.parameters():
    param.requires_grad = False


/gris/gris-f/homestud/jplapper/anaconda3/envs/signenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /gris/gris-f/homestud/jplapper/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:02<00:00, 88.5MB/s] 


In [13]:
# Adjust the final layer according to the number of classes in your dataset
num_ftrs_resnet = resnet101.fc.in_features
NUM_CLASSES = 24
resnet101.fc = nn.Linear(num_ftrs_resnet, NUM_CLASSES)

# Transfer the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet101 = resnet101.to(device)


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet101.fc.parameters(), lr=0.001)

EPOCHS = 10

for epoch in range(EPOCHS):
    resnet101.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_resnet.zero_grad()
        outputs = resnet101(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_resnet.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Training loss: {total_loss/len(train_loader)}")

    # Validation accuracy
    resnet101.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet101(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Validation accuracy: {100.0 * correct / total}%")


Epoch 1/10, Training loss: 2.0801115542087913
Epoch 1/10, Validation accuracy: 64.89046773238603%
Epoch 2/10, Training loss: 1.1397516018939469
Epoch 2/10, Validation accuracy: 80.69863824748371%
Epoch 3/10, Training loss: 0.8339551124932632
Epoch 3/10, Validation accuracy: 83.30373001776199%
Epoch 4/10, Training loss: 0.6618601215897866
Epoch 4/10, Validation accuracy: 85.7312018946122%
Epoch 5/10, Training loss: 0.5606912377308
Epoch 5/10, Validation accuracy: 87.56660746003553%
Epoch 6/10, Training loss: 0.4821411219407927
Epoch 6/10, Validation accuracy: 88.86915334517467%
Epoch 7/10, Training loss: 0.43507545005600406
Epoch 7/10, Validation accuracy: 88.57312018946122%
Epoch 8/10, Training loss: 0.377035675605513
Epoch 8/10, Validation accuracy: 89.81645944345767%
Epoch 9/10, Training loss: 0.35328672348328355
Epoch 9/10, Validation accuracy: 90.34931912374186%
Epoch 10/10, Training loss: 0.32240408082615657
Epoch 10/10, Validation accuracy: 91.71107164002369%


In [15]:
torch.save(resnet101.state_dict(), 'asl_recognition_resnet101.pth')
